In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn
from sklearn.metrics import classification_report
import scikitplot
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/android-ransomware/Android_Ransomeware_Binary_Class_CC3.csv')
data.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1.720000e+26,10420211.0,51023,1.722172e+09,443,6,151054,6,8,1076,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,1.720000e+26,10420211.0,51023,1.722172e+09,443,6,349,2,0,23,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,1.720000e+27,10420211.0,34259,1.722171e+10,443,6,119,2,0,23,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,1.720000e+26,10420211.0,55509,1.722171e+09,443,6,37055,1,1,31,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,1.720000e+26,10420211.0,44852,1.722172e+09,443,6,178727,6,7,1313,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [3]:
data.info()
print("------------------------")
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392034 entries, 0 to 392033
Data columns (total 72 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Flow ID                       392034 non-null  float64
 1    Source IP                    392034 non-null  float64
 2    Source Port                  392034 non-null  int64  
 3    Destination IP               392034 non-null  float64
 4    Destination Port             392034 non-null  int64  
 5    Protocol                     392034 non-null  int64  
 6    Flow Duration                392034 non-null  int64  
 7    Total Fwd Packets            392034 non-null  int64  
 8    Total Backward Packets       392034 non-null  int64  
 9   Total Length of Fwd Packets   392034 non-null  int64  
 10   Total Length of Bwd Packets  392034 non-null  int64  
 11   Fwd Packet Length Max        392034 non-null  int64  
 12   Fwd Packet Length Min        392034 non-nul

Flow ID              0
 Source IP           0
 Source Port         0
 Destination IP      0
 Destination Port    0
                    ..
Idle Mean            0
 Idle Std            0
 Idle Max            0
 Idle Min            0
Label                0
Length: 72, dtype: int64

In [4]:
data['Label'].value_counts()

Ransomware    348943
Benign         43091
Name: Label, dtype: int64

In [5]:
data['Label']= data['Label'].astype('category')
data['Label']= data['Label'].cat.codes

data['Flow ID']= data['Flow ID'].astype('category')
data['Flow ID']= data['Flow ID'].cat.codes

data[' Source IP']= data[' Source IP'].astype('category')
data[' Source IP']= data[' Source IP'].cat.codes

data[' Destination IP']= data[' Destination IP'].astype('category')
data[' Destination IP']= data[' Destination IP'].cat.codes

In [6]:
data['Label'].value_counts()

1    348943
0     43091
Name: Label, dtype: int64

In [7]:
data.corr().style.background_gradient(cmap="Blues")

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
Flow ID,1.000000,0.162680,0.161341,0.720027,0.120891,-0.366145,0.082055,0.047425,0.024509,0.020606,0.016372,0.102164,-0.016818,0.107445,0.123364,0.116331,-0.372140,0.051993,0.150563,0.057482,0.013398,0.021610,0.097339,0.099154,-0.012474,0.069355,0.051619,0.089791,0.093349,0.015822,0.106951,0.095483,0.087379,0.095756,0.054064,0.097568,-0.000690,-0.000685,0.013265,0.005315,-0.152511,0.119973,0.106593,0.123991,0.104467,0.023125,0.097568,0.140691,0.197009,0.114686,-0.153433,0.085716,0.107445,0.051993,-0.000690,0.047425,0.020606,0.024509,0.016372,0.148444,0.019179,0.046158,-0.000227,0.047042,0.018583,0.042536,0.045575,0.068037,-0.000687,0.065048,0.069848,-0.004922
Source IP,0.162680,1.000000,-0.623098,-0.329393,0.820642,-0.158942,-0.118069,-0.040043,-0.025918,0.004315,-0.022700,-0.086593,0.110480,-0.011322,-0.105033,-0.182069,-0.122318,-0.171915,-0.167914,0.028877,0.122603,-0.082801,-0.079571,-0.120914,-0.061879,-0.104787,-0.087859,-0.096306,-0.112539,-0.048983,-0.061205,-0.002609,-0.038346,-0.060902,0.005188,0.090155,0.000878,0.000999,0.142123,-0.028864,-0.117919,-0.142019,-0.137795,-0.149719,-0.096946,-0.007714,0.090155,-0.213744,0.354445,0.321574,0.118440,-0.141991,-0.011322,-0.171915,0.000878,-0.040043,0.004315,-0.025918,-0.022700,-0.216076,0.054327,-0.013687,0.001047,-0.036117,-0.008590,-0.031207,-0.037242,-0.071022,-0.028224,-0.072144,-0.068267,-0.002973
Source Port,0.161341,-0.623098,1.000000,0.488931,-0.677404,-0.106988,0.075668,0.059522,0.039967,0.010522,0.032650,0.145099,-0.115173,0.075348,0.175477,0.239914,-0.100481,0.198254,0.246889,-0.025453,-0.093537,0.024579,0.097319,0.131028,0.002526,0.032211,0.042817,0.099897,0.096749,0.004204,0.109405,0.039448,0.084000,0.108327,0.010073,-0.039768,-0.000653,-0.000446,-0.113546,0.041135,-0.086664,0.207982,0.189619,0.226059,0.163867,0.041532,-0.039768,0.312577,-0.176135,-0.223105,-0.178175,0.180897,0.075348,0.198254,-0.000653,0.059522,0.010522,0.039967,0.032650,0.321242,-0.035462,0.020120,-0.000300,0.032367,-0.011172,0.019720,0.043655,0.055167,-0.004697,0.050892,0.058162,0.001953
Destination IP,0.720027,-0.329393,0.488931,1.000000,-0.353442,-0.461155,0.128456,0.061944,0.040392,0.010489,0.033547,0.157612,-0.131497,0.105551,0.212652,0.219841,-0.392182,0.146861,0.261049,0.016013,-0.040312,0.092780,0.140240,0.165448,0.047833,0.088819,0.106399,0.125785,0.134747,0.055357,0.117711,0.076542,0.095591,0.115314,0.038550,0.009803,-0.000293,-0.000189,-0.051426,0.027035,-0.237575,0.205603,0.176369,0.221948,0.174980,0.029142,0.009803,0.348952,0.072918,-0.028846,-0.238388,0.151923,0.105551,0.146861,-0.000293,0.061944,0.010489,0.040392,0.033547,0.354912,0.010775,0.023786,0.000097,0.043872,0.001904,0.033720,0.050390,0.063588,-0.009762,0.058811,0.066962,0.022366
Destination Port,0.120891,0.820642,-0.677404,-0.353442,1.000000,-0.138219,-0.112308,-0.037814,-0.02609

In [8]:
y = data.Label
X = data.drop('Label', axis=1).values

In [9]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='not minority')
X, y = undersample.fit_resample(X, y)

In [10]:
y.value_counts()

0    43091
1    43091
Name: Label, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 0, stratify = y)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier

# Select the optimal features
selected_features = [0,1,2, 3, 4, 5,6,11,12,16,59,60,62,63,64,65,67,68]  

# Subset the input features with the selected features
X_selected = X[:, selected_features]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Build the ANN model with the selected features
#model = Sequential()
#model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
#model.add(Dense(1, activation='sigmoid'))
model = RandomForestClassifier()
model.fit(X_train, y_train)
# Compile and train the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the test set
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9390845274699774


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Load your data and select the desired features
selected_features = [0,1,2, 3, 4, 5,6,13,14,16,17,18,19,20,26,29,31,36 ,39,44,55,56,59,58,60,62,63,64,65,67,68,69]  
X_selected = X[:, selected_features]  # If X is a numpy array
# X_selected = X[selected_features]  # If X is a pandas DataFrame

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the Gradient Boosting Tree model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8630852236468063


In [15]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your data and select the desired features
# X: input features (numpy array or pandas DataFrame) containing all features
# selected_features: list of indices or column names of the desired features
selected_features = [0,1,2, 3, 4, 5,6,13,14,16,17,18,19,20,26,29,31,36 ,39,44,55,56,59,58,60,62,63,64,65,67,68,69]  
X_selected = X[:, selected_features]  # If X is a numpy array
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=40)

# Convert the data to DMatrix format (required by XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define the parameters for the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'error'
}
# Train the XGBoost model
model = xgb.train(params, dtrain)

# Make predictions on the test set
y_pred_proba = model.predict(dtest)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8626211057608633


In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load your data and select the desired features
selected_features = [0,1,2, 3, 4, 5,6,11,12,16,59,60,61,62,63,64,65,67,68]  
X_selected = X[:, selected_features]  # If X is a numpy array
# X_selected = X[selected_features]  # If X is a pandas DataFrame

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the Decision Tree model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.942623426350293


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your data and select the desired features
selected_features = [0,1,2, 3, 4, 5,6,11,12,16,59,60,61,62,63,64,65,67,68]  
X_selected = X[:, selected_features]  # If X is a numpy array
# X_selected = X[selected_features]  # If X is a pandas DataFrame

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)

# Set up the LightGBM parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
}

# Train the LightGBM model
model = lgb.train(params, train_data)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)  # Convert probabilities to binary predictions

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load your data and select the desired features
# X: input features (numpy array or pandas DataFrame) containing all features
# selected_features: list of indices or column names of the desired features
selected_features = [0,1,2, 3, 4, 5,6,11,12,16,59,60,61,62,63,64,65,67,68]  

X_selected = X[:, selected_features]  # If X is a numpy array
# X_selected = X[selected_features]  # If X is a pandas DataFrame

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Train the XGBoost model
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set using both models
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

# Combine the predictions using a simple voting mechanism
hybrid_predictions = np.round((rf_predictions + xgb_predictions) / 2)

# Calculate accuracy
accuracy = accuracy_score(y_test, hybrid_predictions)
print("Accuracy:", accuracy)
